In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
from keras.preprocessing import image
from sklearn import metrics

In [ ]:
#Dataset link=https://drive.google.com/drive/folders/1gmsoDAoXUIihitinvjPlg67DJP_iGs8k?usp=sharing
train_path1='/content/drive/MyDrive/image_dataset/train/planes/'
train_path2='/content/drive/MyDrive/image_dataset/train/bike/'
test_path1='/content/drive/MyDrive/image_dataset/test/planes/'
test_path2='/content/drive/MyDrive/image_dataset/test/bike/'

In [ ]:
train_x=np.zeros((12288,1500))
train_y=np.zeros((1,1500))
test_x=np.zeros((12288,98))
test_y=np.zeros((1,98))

In [ ]:
train_files1=os.listdir(train_path1)
train_files2=os.listdir(train_path2)
for i in range(len(train_files1)):
  img=image.load_img(train_path1+train_files1[i],target_size=(64,64))
  arr=np.array(img)
  arr=arr.flatten()
  train_x[:,i]=arr
  train_y[0,i]=0
for i in range(len(train_files2)):
  img=image.load_img(train_path2+train_files2[i],target_size=(64,64))
  arr=np.array(img)
  arr=arr.flatten()
  train_x[:,i+750]=arr
  train_y[0,i+750]=1

In [ ]:
test_files1=os.listdir(test_path1)
test_files2=os.listdir(test_path2)
for i in range(len(test_files1)):
  img=image.load_img(test_path1+test_files1[i],target_size=(64,64))
  arr=np.array(img)
  arr=arr.flatten()
  test_x[:,i]=arr
  test_y[0,i]=0
for i in range(len(test_files2)):
  img=image.load_img(test_path2+test_files2[i],target_size=(64,64))
  arr=np.array(img)
  arr=arr.flatten()
  test_x[:,i+50]=arr
  test_y[0,i+50]=1

In [ ]:
train_x=train_x/255.0
test_x=test_x/255.0

In [ ]:
def sigmoid(z):
  return (1/(1+np.exp(-z)))

In [ ]:
def init_params(dimension):
  w=np.zeros((dimension,1))
  b=0
  return w,b

In [ ]:
def propagate(w,b,X,Y):
  n=X.shape[1] #number of training examples
  
  #forward pass
  predicted=sigmoid(np.dot(w.T,X)+b)
  cost=(-1/n)*np.sum(np.multiply(Y,np.log(predicted))+np.multiply((1-Y),np.log(1-predicted)))
  
  #backward pass
  dw=(1/n)*np.dot(X,(predicted-Y).T)
  db=(1/n)*np.sum(predicted-Y)
  
  cost=np.squeeze(cost)

  #grad dictionary
  grads={"dw":dw,"db":db}

  return grads,cost


In [ ]:
def optimize(w,b,X,Y,epochs,lr):
  costs=[]
  for i in range(epochs):
    grads,cost=propagate(w,b,X,Y)

    dw=grads["dw"]
    db=grads["db"]

    w=w-(lr*dw)
    b=b-(lr*db)

    if(i%10==0):
      costs.append(cost)
      print("cost after %i iterations:%f"%(i,cost))
  
  params={"w":w,"b":b}
  grads={"dw":dw,"db":db}

  return params,grads,costs



In [ ]:
def predict(w,b,X):
  Y_predict=np.zeros((1,X.shape[1]))
  A=sigmoid(np.dot(w.T,X)+b)
  for i in range(A.shape[1]):
    if(A[0,i]<=0.5):
      Y_predict[0,i]=0
    else:
      Y_predict[0,i]=1
  return Y_predict


In [ ]:
def model(X_train,Y_train,X_test,Y_test,epochs,lr):
  w,b=init_params(X_train.shape[0])
  params,grads,costs=optimize(w,b,X_train,Y_train,epochs,lr)
  w=params["w"]
  b=params["b"]
  train_predict=predict(w,b,X_train)
  test_predict=predict(w,b,X_test)
  print("train_accuracy: {} %".format(100-np.mean(np.abs(train_predict - Y_train)) * 100))
  print("test_accuracy : {} %".format(100-np.mean(np.abs(test_predict  - Y_test)) * 100))

In [ ]:
model(train_x,train_y,test_x,test_y,100,0.001)

cost after 0 iterations:0.693147
cost after 10 iterations:0.547769
cost after 20 iterations:0.464781
cost after 30 iterations:0.410535
cost after 40 iterations:0.372720
cost after 50 iterations:0.344937
cost after 60 iterations:0.323652
cost after 70 iterations:0.306789
cost after 80 iterations:0.293061
cost after 90 iterations:0.281633
train_accuracy: 92.4 %
test_accuracy : 100.0 %


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
model=keras.Sequential()
model.add(keras.layers.Dense(1,activation='sigmoid',input_shape=(12288,)))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 12289     
                                                                 
Total params: 12,289
Trainable params: 12,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
model.fit(train_x.T,train_y.flatten(),epochs=20)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


47/47 [==============================] - 1s 2ms/step - loss: 1.8506 - accuracy: 0.7187
Epoch 2/20
47/47 [==============================] - 0s 2ms/step - loss: 0.2289 - accuracy: 0.9253
Epoch 3/20
47/47 [==============================] - 0s 2ms/step - loss: 0.3938 - accuracy: 0.8893
Epoch 4/20
47/47 [==============================] - 0s 2ms/step - loss: 0.1687 - accuracy: 0.9507
Epoch 5/20
47/47 [==============================] - 0s 2ms/step - loss: 0.1404 - accuracy: 0.9480
Epoch 6/20
47/47 [==============================] - 0s 2ms/step - loss: 0.4481 - accuracy: 0.8867
Epoch 7/20
47/47 [==============================] - 0s 2ms/step - loss: 0.1033 - accuracy: 0.9640
Epoch 8/20
47/47 [==============================] - 0s 2ms/step - loss: 0.0987 - accuracy: 0.9620
Epoch 9/20
47/47 [==============================] - 0s 2ms/step - loss: 0.0865 - accuracy: 0.9687
Epoch 10/20
47/47 [==============================] - 0s 2ms/step - loss: 0.0692 - accuracy: 0.9753
Epoch 11/20
47/47 [===========

In [ ]:
model.evaluate(test_x.T,test_y.flatten())

4/4 [==============================] - 0s 3ms/step - loss: 0.0352 - accuracy: 0.9898


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


[0.035177215933799744, 0.9897959232330322]